<a href="https://colab.research.google.com/github/djamoreland/Quantitative-Neuroscience-Moreland/blob/main/Multiple_Comparisons.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import t
import scipy.stats as st
import pandas as pd

In this exercise we will run through an example of correcting for multiple comparisons with both the Benjamini-Hochberg procedure and the more conservative Bonferroni correction.

1. First, simulate multiple (say, 1000) t-tests comparing two samples with equal means and standard deviations, and save the p-values. Obviously, at p<0.05 we expect that ~5% of the simulations to yield a "statistically significant" result (of rejecting the NULL hypothesis that the samples come from distributions with equal means).


2. Second, once you have the simulated p-values, apply both methods to address the multiple comparisons problem.

3. Third, set the sample 1 and sample 2 means to be 1 and 2 respectively, and re-run the exercise. What do you notice? What if you make the difference between means even greater?

In [3]:
#simulate t-tests

#Create samples with mean of 9.22 and std of idk
mean = 9.22
std = 1.03
n=100

#Run 1000 t-test for those two samples

N_tests = 1000
p_values = []

for i in range(N_tests):
    sample1 = t.rvs(n-1, loc=mean, scale=std, size=n)
    sample2 = t.rvs(n-1, loc=mean, scale=std, size=n)
    t_stat, p_value = st.ttest_ind(sample1, sample2)

#Save p-values
    p_values.append(p_value)


In [10]:
#Apply multiple comparisons

#Bonferroni
alpha = 0.05

corrected_alpha = alpha/N_tests
print(f'The Bonferroni corrected threshold is {corrected_alpha}')

#Benjamini-Hochberg
Q = 0.05

df = pd.DataFrame()
df['p-values'] = p_values
df = df.sort_values('p-values').reset_index(drop=True)
df['rank'] = np.arange(1, N_tests + 1)

df['crit_value'] = (df['rank'] / N_tests) * Q

# Find the largest p-value that is less than or equal to its critical value
significant_p_values = df.loc[df['p-values'] <= df['crit_value'], 'p-values']

if not significant_p_values.empty:
    BH_threshold = significant_p_values.max()
    print(f'The Benjamini-Hochberg corrected threshold is {BH_threshold}')
else:
    BH_threshold = 0 # No significant results found
    print('No p-values were below their Benjamini-Hochberg critical values. No significant results at the Q=0.05 level.')

The Bonferroni corrected threshold is 5e-05
No p-values were below their Benjamini-Hochberg critical values. No significant results at the Q=0.05 level.


In [12]:
#art 3
#Create samples with mean of 9.22 and std of idk
std = 1.03
n=100

#Run 1000 t-test for those two samples

N_tests = 1000
p_values = []

for i in range(N_tests):
    sample1 = t.rvs(n-1, loc=1, scale=std, size=n)
    sample2 = t.rvs(n-1, loc=2, scale=std, size=n)
    t_stat, p_value = st.ttest_ind(sample1, sample2)

#Save p-values
    p_values.append(p_value)



#Apply multiple comparisons

#Bonferroni
alpha = 0.05
corrected_p_value = alpha/N_tests
#corrected_p_value = [p_values[j]/N_tests for j in range(len(p_values))
print(f'The Bonferroni corrected threshold is {corrected_p_value}')

#Benjamini-Hochberg
Q = 0.05
rank = []

#Create Ranks
r= 1
for _ in range(N_tests):
  rank.append(r)
  r+=1

#Find critical values
crit_value = []
for k in range(len(rank)):
  crit = (rank[k]/N_tests) * Q
  crit_value.append(crit)


#find largest value smaller than its crit value
df = pd.DataFrame()
df['p-values'] = p_values
df['rank'] = rank
df['crit_value'] = crit_value
BH_corrected_p = df[df['p-values'] < df['crit_value']]['p-values'].max()

print(f'The corrected threshold after a Benjamini-Hochberg correction is {BH_corrected_p}')

The Bonferroni corrected threshold is 5e-05
The corrected threshold after a Benjamini-Hochberg correction is 0.00016485439145806687


As the distance between the means increase the corrected threshold becomes smaller

In [14]:
#With help of chatgpt

# Parameters
mean1 = 1
mean2 = 2   # different mean so we can see some effect
std = 1.03
n = 100
N_tests = 1000
alpha = 0.05
Q = 0.05  # FDR

# Simulate p-values from independent two-sample t-tests
p_values = []
for i in range(N_tests):
    sample1 = np.random.normal(loc=mean1, scale=std, size=n)
    sample2 = np.random.normal(loc=mean2, scale=std, size=n)
    t_stat, p_value = st.ttest_ind(sample1, sample2)
    p_values.append(p_value)

# --- Bonferroni correction ---
bonf_threshold = alpha / N_tests
print(f"Bonferroni corrected threshold: {bonf_threshold:.6f}")

# --- Benjamini–Hochberg correction ---
df = pd.DataFrame({"p": p_values})
df = df.sort_values("p").reset_index(drop=True)
df["rank"] = np.arange(1, N_tests+1)
df["crit"] = (df["rank"] / N_tests) * Q

# Largest p-value under its critical value
bh_threshold = df.loc[df["p"] <= df["crit"], "p"].max()

print(f"Benjamini–Hochberg corrected threshold: {bh_threshold:.6f}")

# Mark significance
df["significant"] = df["p"] <= bh_threshold


Bonferroni corrected threshold: 0.000050
Benjamini–Hochberg corrected threshold: 0.000132
